<a href="https://colab.research.google.com/github/qkrjuyeol/boiler-efficiency-checker/blob/main/AI/Data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 데이터 전처리 - 아스카로바

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# 데이터 폴더 경로
from google.colab import drive
drive.mount('/content/drive')

data_folder = '/content/drive/MyDrive/3months_data'  # Update with the correct path


Mounted at /content/drive


In [2]:
all_files = os.listdir(data_folder)
csv_files = [os.path.join(data_folder, f) for f in all_files if f.endswith('.csv')]


#가능한 모든 인코딩
def read_csv_file(file_path):
    encodings = ['utf-8', 'cp949', 'euc-kr']  # List of possible encodings
    for encoding in encodings:
        try:
            return pd.read_csv(file_path, encoding=encoding)
        except (UnicodeDecodeError, pd.errors.EmptyDataError, pd.errors.ParserError):
            continue
    raise ValueError(f"Could not read file {file_path} with any encoding.")


valid_dataframes = []
#모든 파일을 포함한 dataframe 만들기
for file in csv_files:
    try:
        df = read_csv_file(file)
        if not df.empty:  # Check if the DataFrame has data
            print(f"File '{file}' read successfully with {len(df)} rows.")
            valid_dataframes.append(df)
        else:
            print(f"File '{file}' is empty and will be skipped.")
    except ValueError as e:
        print(e)
#empty 파일 제외한 모든 파일을 dataframe으로 추가하기

# Combine only the valid DataFrames
if valid_dataframes:
    combined_data = pd.concat(valid_dataframes, ignore_index=True)
else:
    combined_data = pd.DataFrame()  # Empty DataFrame if no valid files found

# Verify the combined DataFrame
print(f"Total rows in combined data: {len(combined_data)}")
print(combined_data.head())



File '/content/drive/MyDrive/3months_data/28_2024-06-15.csv' read successfully with 34817 rows.
File '/content/drive/MyDrive/3months_data/28_2024-06-16.csv' read successfully with 34739 rows.
File '/content/drive/MyDrive/3months_data/28_2024-06-17.csv' read successfully with 34667 rows.
File '/content/drive/MyDrive/3months_data/28_2024-06-18.csv' read successfully with 34718 rows.
File '/content/drive/MyDrive/3months_data/28_2024-06-19.csv' read successfully with 34666 rows.
File '/content/drive/MyDrive/3months_data/28_2024-06-20.csv' read successfully with 34631 rows.
File '/content/drive/MyDrive/3months_data/28_2024-06-21.csv' read successfully with 34580 rows.
File '/content/drive/MyDrive/3months_data/28_2024-06-22.csv' read successfully with 34526 rows.
File '/content/drive/MyDrive/3months_data/28_2024-06-23.csv' read successfully with 15481 rows.
File '/content/drive/MyDrive/3months_data/28_2024-06-24.csv' read successfully with 34412 rows.
File '/content/drive/MyDrive/3months_dat

In [3]:
combined_data = combined_data.drop(columns=[
    "생성일",
    "소비전류",
    "진동센서1",
    "진동센서2",
    "운전시간",
    "정상 운전 확률",
    "송풍기 고장 확률",
    "AIR 댐퍼 고장 확률",
    "GAS 앰퍼 고장 확률",
    "확률 업데이트 시간",
    "순간 스팀량",
    "입출력법 효율",
    "열 손실법 효율"

])

# Scale numerical features (optional, but good practice)
scaler = StandardScaler()
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

combined_data.fillna(method='ffill', inplace=True)  # Forward fill missing values
combined_data.fillna(method='bfill', inplace=True)  # Backward fill for any remaining NaNs

# Select numerical columns only
numerical_columns = combined_data.select_dtypes(include=['float64', 'int64']).columns

# Fill missing values in numerical columns with the mean value
combined_data[numerical_columns] = combined_data[numerical_columns].apply(lambda col: col.fillna(col.mean()))

# For categorical columns, fill missing values with the most frequent value
cat_imputer = SimpleImputer(strategy='most_frequent')
df[df.select_dtypes(include=['object']).columns] = cat_imputer.fit_transform(df.select_dtypes(include=['object']))

# Encode categorical variables (if any)
label_encoder = LabelEncoder()
for column in df.select_dtypes(include=['object']).columns:
    df[column] = label_encoder.fit_transform(df[column])




/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
<ipython-input-3-4f2a53092930>:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_data.fillna(method='ffill', inplace=True)  # Forward fill missing values
<ipython-input-3-4f2a53092930>:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_data.fillna(method='bfill', inplace=

In [4]:
combined_data.to_csv('/content/drive/MyDrive/preprocessed_boiler_data.csv', index=False, encoding='utf-8')
combined_data.head()
#checking if data preprocessing was correct
print(combined_data.isna().sum())  # Shows the count of NaNs in each column
print(combined_data.isna().sum().sum())  # Total count of missing values
print(combined_data.info())


부하율                   0
설정 압력                 0
보일러 압력                0
송풍기 인버터 출력            0
송풍기 입력                0
급수 펌프                 0
급수펌프 입력               0
가스 댐퍼                 0
가스 댐퍼 입력              0
Air 댐퍼                0
Air 댐퍼 입력             0
재순환 댐퍼                0
재순환 외기 댐퍼             0
재순환 댐퍼 입력             0
재순환 외기 댐퍼 입력          0
급수 수위                 0
보일러 온도                0
배기가스온도1               0
배기가스온도2               0
배기가스온도3               0
배기 재 순환 온도            0
에코 온도1                0
에코 온도2                0
버너온도                  0
배기가스 NOx              0
배기가스 O2               0
재순환 O2                0
재순환 NOx               0
급수량(적산유량)             0
급수량(순간유량)             0
연료량(적산유량)             0
연료량(순간유량)             0
효율(순간)                0
효율(입출력법-스팀)     4080114
dtype: int64
4080114
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4080114 entries, 0 to 4080113
Data columns (total 34 columns):
 #   Column        Dtype  
---  ------        ----